# Optimization and Simulation with DR

## ランダムサーチによる単目的関数の最適化

### 1. ライブラリ類のインポートとファイルのロード

In [ ]:
%matplotlib inline

import os
import warnings
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import stats
import matplotlib.pyplot as plt
from pandas import DataFrame
from scipy.interpolate import griddata
import datarobot as dr

import japanize_matplotlib

if not os.getenv("DATAROBOT_NOTEBOOK_IMAGE"):
    print("not running in DataRobot Notebook")
    from dotenv import load_dotenv
    load_dotenv("../.env", override=True)

client = dr.Client()

seed = 71
np.random.seed(seed)
# sns.set_theme(style='darkgrid')
warnings.filterwarnings("ignore")

pd.set_option("display.max_rows", 1000)
pd.set_option("display.max_columns", 1000)
pd.set_option("display.width", 1000)
pd.set_option("display.max_colwidth", 1000)
pd.set_option("display.precision", 8)

In [ ]:
deployment_id = "67bc6ab2f1b3f6b73d560ff8"
df = pd.read_csv("../data/opt_steel_strength.csv")

In [ ]:
y_train = df["降伏強度"]
X_train = df.drop(["ID", "降伏強度", "引張強度"], axis=1)

In [ ]:
# 312rows, 16columns
df.shape

In [ ]:
df.head()

### 2. 探索空間の指定とデータの生成

In [ ]:
# 正規分布の上下を元データのmin, maxの少しだけ外側でトリムします。
n = 1000
X_artificial = DataFrame(index=range(n), columns=X_train.columns)

for col in X_artificial.columns:

    norm_dist = np.random.normal(X_train[col].mean(), X_train[col].std(), size=n)
    min_value = X_train[col].min()
    max_value = X_train[col].max()
    norm_dist = np.clip(norm_dist, min_value * 0.8, max_value * 1.2)

    X_artificial.loc[:, col] = norm_dist

X_artificial = pd.to_numeric(X_artificial.stack(), errors="coerce").unstack()

In [ ]:
# csvに保存して、DataRobotで予測を行ってみましょう。
X_artificial.to_csv(
    "../data/opt_steel_strength_artificial_1000.csv", index=False, encoding="utf-8-sig"
)  # , encoding='SJIS'

### 予測
生成したDataFrameを用いて、DataRobotのバッチ予測で降伏強度を予測してみましょう。<br>
結果をExcelやpandas等でソートすることで、最も強度が高くなる条件が求まります。

In [ ]:
_, df_result = dr.models.BatchPredictionJob.score_pandas(
    deployment=deployment_id, df=X_artificial
)

In [ ]:
max_index = df_result["降伏強度_PREDICTION"].idxmax()
max_value = df_result["降伏強度_PREDICTION"].max()

df_result["降伏強度_PREDICTION"].plot(color="blue")
plt.scatter(max_index, max_value, color="red", marker="o")
plt.title(f"予測した降伏強度 最大値: {max_value:.2f}")
plt.xlabel("試行回数")
plt.show()

In [ ]:
%%time
# 10,000件に増やしてもう一回やってみます。
n = 10000
X_artificial = DataFrame(index=range(n),
                         columns=X_train.columns)

for col in X_artificial.columns:

    norm_dist = np.random.normal(X_train[col].mean(), X_train[col].std(), size=n)
    min_value = X_train[col].min()
    max_value = X_train[col].max()
    norm_dist = np.clip(norm_dist, min_value * 0.8, max_value * 1.2)

    X_artificial.loc[:, col] = norm_dist
        
X_artificial = pd.to_numeric(X_artificial.stack(), errors='coerce').unstack()

In [ ]:
X_artificial.to_csv(
    "../data/opt_steel_strength_artificial_10000.csv",
    index=False,
    encoding="utf-8-sig",
)  # , encoding='SJIS'

### 得られたデータ点の分布と予測結果の分布を可視化してみましょう

In [ ]:
# 元のデータの分布をみてみましょう。ここではチタンとアルミニウムについて可視化してみます。
feat_1 = "チタン"

feat_2 = "アルミニウム"

r, p = stats.pearsonr(X_train[feat_1], X_train[feat_2])
g = sns.JointGrid(data=X_train, x=feat_1, y=feat_2)
g.plot_joint(sns.kdeplot, fill=True, color="r")
g.plot_joint(plt.scatter, c="r", s=0.01)
g.plot_marginals(sns.kdeplot, color="r")
g.ax_joint.text(x=1.5, y=2, s=f"pearsonr={r:.2f}, p={p:.2e}")
g.set_axis_labels(feat_1, feat_2)

plt.show()

In [ ]:
# 正規分布から生成したデータの分布をみてみましょう。
r, p = stats.pearsonr(X_artificial[feat_1], X_artificial[feat_2])
g = sns.JointGrid(data=X_artificial, x=feat_1, y=feat_2)
g.plot_joint(sns.kdeplot, fill=True, color="r")
g.plot_joint(plt.scatter, c="r", s=0.01)
g.plot_marginals(sns.kdeplot, color="r")
g.ax_joint.text(x=1.5, y=2, s=f"pearsonr={r:.4f}, p={p:.2F}")
g.set_axis_labels(feat_1, feat_2)

plt.show()

In [ ]:
_, df_result = dr.models.BatchPredictionJob.score_pandas(
    deployment=deployment_id, df=X_artificial
)

In [ ]:
max_index = df_result["降伏強度_PREDICTION"].idxmax()
max_value = df_result["降伏強度_PREDICTION"].max()

df_result["降伏強度_PREDICTION"].plot(color="blue")
plt.scatter(max_index, max_value, color="red", marker="o")
plt.title(f"予測した降伏強度 最大値: {max_value:.2f}")
plt.xlabel("試行回数")
plt.show()

In [ ]:
df_result.head()

In [ ]:
# 特徴空間（2D）における強度予測値の分布を可視化してみます
x = np.linspace(X_artificial[feat_1].min(), X_artificial[feat_1].max(), 10)
y = np.linspace(X_artificial[feat_2].min(), X_artificial[feat_2].max(), 10)

z = griddata(
    (X_artificial[feat_1].values, X_artificial[feat_2].values),
    df_result["降伏強度_PREDICTION"].values,
    (x[None, :], y[:, None]),
    method="linear",
)

plt.figure(figsize=[7, 7])
plt.contourf(x, y, z, 5, alpha=0.6, cmap=plt.cm.magma)
plt.colorbar()
plt.clim(df_result["降伏強度_PREDICTION"].min(), df_result["降伏強度_PREDICTION"].max())
plt.xlabel(feat_1)
plt.ylabel(feat_2)
plt.title("強度予測値の分布")
plt.show()

In [ ]:
plt.figure(figsize=[7, 7])
plt.plot(x, np.nanmean(z, axis=0))
plt.xlabel(feat_1)
plt.ylabel("降伏強度")
plt.show()

In [ ]:
plt.figure(figsize=[7, 7])
plt.plot(y, np.nanmean(z, axis=1))
plt.xlabel(feat_2)
plt.ylabel("降伏強度")
plt.show()